In [1]:
import cloudComPy as cc
import cloudComPy.RANSAC_SD
import trimesh
import igl
import numpy as np
import pickle
import os
from multiprocessing import Pool
import time
from Py_RMSE_Sphere import SDF_individual_sphere

# Define paths

In [ ]:
Dir_main='PATH' #path for importing STL files (scanned dental arches). Files in the directory must be named using numbers: '1.stl', '2.stl', etc.
Path_to_save="PATH" #path for saving pickle files


# Parameters to tune

In [2]:
#------------------- RANSAC parametrai (change accordingly) --------------------------
#parameters were set to be compatible with 'Arch_model_2.5mm.stl'

params = cc.RANSAC_SD.RansacParams()

params.allowSimplification=False
params.maxSphereRadius=2.6
params.minSphereRadius=2.4
params.maxNormalDev_deg=20
params.setPrimEnabled(cc.RANSAC_SD.RPT_PLANE,False)
params.setPrimEnabled(cc.RANSAC_SD.RPT_CYLINDER,False)

params.epsilon=0.1 
params.bitmapEpsilon=0.2

# Main

In [ ]:
start_time = time.time()

for igg,file_to_import in enumerate([f for f in os.listdir(Dir_main) if f.endswith('.stl')]):
    Reduces=trimesh.load(os.path.join(Dir_main,file_to_import))

    cloud = cc.ccPointCloud()
    cloud.coordsFromNPArray_copy(Reduces.vertices)
    cc.computeNormals([cloud])
    cloud.normalsFromNpArrayCopy(Reduces.vertex_normals.copy(order='C'))  
   
    #[cc.RANSAC_SD.RPT_CONE,cc.RANSAC_SD.RPT_PLANE,cc.RANSAC_SD.RPT_SPHERE,cc.RANSAC_SD.RPT_CYLINDER,cc.RANSAC_SD.RPT_TORUS]
    
    MEGA_Store_R=[]
    MEGA_Store_SD_outlier=[] 
    MEGA_Store_SD_sphers=[]
    
    for index_loop,sample_size in zip(range(20),np.arange(150,250,5)): #Running RANSAC multiple times to reduce stochasticity
        params.supportPoints=sample_size
        meshes, clouds = cc.RANSAC_SD.computeRANSAC_SD(cloud, params)
        Store_Sphere_R_OG=[] # Radius fitted sphere
        Store_Sphere_C_OG=[] # Center fitter sphere
        Store_point_cloud=[] # Center fitter sphere
        
        Store_SD_outlier=[] 
        Store_SD_sphers=[]
        #----------------------------First RANSAC---------------------------------
        for indexx,cl in enumerate(clouds):
           if cl is not None:
            if cl.getName()=="Leftovers":
             Left_over=cl
            else:

             center_OG=meshes[indexx].getAssociatedCloud().computeGravityCenter()
             radius_OG=meshes[indexx].getRadius()
             Store_point_cloud.append(cl.toNpArray())
             Store_Sphere_R_OG.append(radius_OG)
             Store_Sphere_C_OG.append(center_OG)
                   
        Store_Sphere_C_OG=np.array(Store_Sphere_C_OG)
        Store_Sphere_R_OG=np.array(Store_Sphere_R_OG) 
        #----------------Create model form sphere (CSG) and compute SDF------------------
        List_Sphere_created=[trimesh.primitives.Sphere(R_OG,C_OG,subdivisions =4) for C_OG,R_OG in zip(Store_Sphere_C_OG,Store_Sphere_R_OG)]
        Bool_Spheres_real=trimesh.boolean.boolean_manifold(List_Sphere_created,'union') # combine spheres into one model
        # ------------------------Compute RMSE values for each sphere fitted--------------
        with Pool(processes=os.cpu_count()) as pool:
            saved_area = pool.map(SDF_individual_sphere, list(zip(Store_Sphere_R_OG, Store_Sphere_C_OG, Store_point_cloud)))
        _, _, SDF = zip(*saved_area)
        Store_SD_sphers.append(np.hstack(SDF))
        
        # ----Outlier SD------
        store_view = Reduces.vertices.view([('', Reduces.vertices.dtype)] * Reduces.vertices.shape[1])
        reduce_view =  np.vstack(Store_point_cloud).view([('', np.vstack(Store_point_cloud).dtype)] * np.vstack(Store_point_cloud).shape[1])
        unique_rows = np.setdiff1d(store_view, reduce_view)
        unique_rows = unique_rows.view(Reduces.vertices.dtype).reshape(-1, 3)
        
        SDF_outlier, _, _ = igl.signed_distance(unique_rows, Bool_Spheres_real.vertices,Bool_Spheres_real.faces, return_normals=False)
        Store_SD_outlier.append(SDF_outlier[(SDF_outlier<0.2) & (SDF_outlier>-0.2)]) # triming is nesecery for 

        MEGA_Store_R.append(Store_Sphere_R_OG)
        MEGA_Store_SD_outlier.append(Store_SD_outlier)
        MEGA_Store_SD_sphers.append(Store_SD_sphers)
            
    #---DATA for exportation-------
    Export_data={
    "Scan_radius":MEGA_Store_R,
    "SD_outliers":MEGA_Store_SD_outlier,
    "SD_spheres":MEGA_Store_SD_sphers
    }

    if not os.path.exists(Path_to_save):
       os.makedirs(Path_to_save)
    with open(os.path.join(Path_to_save,file_to_import[:-3]+"pkl"), "wb") as f:
       pickle.dump(Export_data, f)
  
end_time = time.time()
print("Time elapsed: %.2f minutes" % ((end_time - start_time) / 60))